In [1]:
import pandas as pd
import plotly_express as px

df = pd.read_csv('yearly_full_release_long_format-1.csv')
df

,Area,Country code,Year,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,YoY absolute change,YoY % change
0,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Clean,GW,0.19,NaN,NaN
1,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Fossil,GW,0.03,NaN,NaN
2,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Gas and Other Fossil,GW,0.03,NaN,NaN
3,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,"Hydro, Bioenergy and Other Renewables",GW,0.19,NaN,NaN
4,Afghanistan,AFG,2000,Country,Asia,Asia,0.0,0.0,0.0,0.0,Capacity,Aggregate fuel,Renewables,GW,0.19,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323525,Zimbabwe,ZWE,2021,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Other Fossil,mtCO2,0.03,0.0,0.00
323526,Zimbabwe,ZWE,2021,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Other Renewables,mtCO2,0.00,0.0,NaN
323527,Zimbabwe,ZWE,2021,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Solar,mtCO2,0.00,0.0,NaN
323528,Zimbabwe,ZWE,2021,Country,Africa,Africa,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Wind,mtCO2,0.00,0.0,NaN
